In [56]:
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain, ConversationChain
from langchain.document_loaders import JSONLoader
from langchain.document_loaders import GoogleDriveLoader
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
# from config import *
import os
import sys
import requests
import json
sys.path.append('./chat')

SALES_GOAL = """provide clear and concise answers to potential customer about his queries about the company, but ultimately convince him to signup."""

SALES_TEMPLATE = """
    Below is a query from a user and
    some relevant contexts. Answer the question given the information in those
    contexts. 

    Content Context: `{context}`

    Conversation Handling:

    Previous Chat: `{chat_history}`

    Last Customer Message: `{human_input}`

"""

PROMPT_TEMPLATE = ChatPromptTemplate.from_template(SALES_TEMPLATE)

class QAChain:
    def __init__(self, folder_id=None):
#         if len(examples) > 0:
#             self.examples_str = "\n\n".join([example['content'] for example in examples])
#         else:
#             self.examples_str = ""
        # Load docs from Google docs
        if folder_id is not None:
            self.folder_id = folder_id
            print("B")
            # Load docs from Google drive
            self.load_docs_from_google()
            print("C")
        else:
            self.docs_google = []
        # Load docs from manual docs on studio
        self.load_docs_from_json()
        # Contatenate all docs
        self.docs = self.docs_json + self.docs_google

        # Create Chroma vectorstore
        self.create_vectorstore()
        print("D")

        # Define qa chain parameters
        self.llm = ChatOpenAI(temperature=0)
        print("E")
        self.memory = ConversationBufferMemory(memory_key="chat_history", input_key="human_input")
        print("F")
        self.prompt = PromptTemplate(
            input_variables=["chat_history", 
                             "human_input", 
                             "context" ], template=SALES_TEMPLATE
            )
        print("G")
        self.qachain = load_qa_chain(
            self.llm, 
            chain_type="stuff", 
            memory=self.memory, 
            prompt=self.prompt
        )
        print("H")

    def get_response_over_docs(self, query):
        relevant_docs = self.retriever.get_relevant_documents(
            query
        )
        return self.qachain(
            {"input_documents": relevant_docs, 
             "human_input": query}, 
            return_only_outputs=True)['output_text']
    
    def load_docs_from_json(self):
        """Load docs from manual input on studio"""
        # Document loading
        self.loader_json = JSONLoader(
            file_path=f'./docs/{self.sessionId}.json',
            jq_schema='.documents[].description')

        self.docs_json = self.loader_json.load()

    def load_docs_from_google(self):
        """Folder should contain Google docs"""
        try:
            # Document loading
            self.loader_google = GoogleDriveLoader(
                folder_id=self.folder_id,
                # Optional: configure whether to recursively fetch files from subfolders. Defaults to False.
                recursive=False,
                credentials_path=".credentials/credentials.json",
                token_path=".credentials/token.json",
            )

            self.docs_google = self.loader_google.load()

        except Exception as e:
            # Check if the error relates to token expiration (adapt based on actual exception message)
            if 'token expired' in str(e).lower():
                refreshed = self.refresh_token()
                if refreshed:
                    self.load_docs_from_google() # Retry loading the documents
                else:
                    raise Exception("Failed to refresh the token.")
            else:
                raise e

    def create_vectorstore(self):
        self.text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=0)
        self.documents = self.text_splitter.split_documents(self.docs)

        self.embeddings = OpenAIEmbeddings()
        self.vectorstore = Chroma.from_documents(self.documents, self.embeddings)
        self.retriever = self.vectorstore.as_retriever(search_kwargs={"k":1})
    
    def refresh_token(self):
        # Load values from credentials.json
        with open('.credentials/credentials.json', 'r') as cred_file:
            cred_data = json.load(cred_file)
        
        # Load refresh_token from token.json
        with open('.credentials/token.json', 'r') as token_file:
            token_data = json.load(token_file)

        # Constructing the refresh URL with the necessary parameters
        params = {
            'client_id': cred_data['installed']['client_id'],
            'client_secret': cred_data['installed']['client_secret'],
            'refresh_token': token_data['refresh_token'],
            'grant_type': 'refresh_token'
        }

        response = requests.post("https://oauth2.googleapis.com/token", data=params)

        if response.status_code == 200:
            new_token = response.json()['access_token']
            
            # Load existing token.json into a dictionary
            with open('.credentials/token.json', 'r') as token_file:
                token_data = json.load(token_file)
            
            # Update the access token
            token_data['token'] = new_token
            
            # Save updated token.json
            with open('.credentials/token.json', 'w') as token_file:
                json.dump(token_data, token_file)

            return True
        else:
            print("Error refreshing token:", response.json())
            return False


INFO:chromadb.db.duckdb:Exiting: Cleaning up .chroma directory


In [57]:
import time
import base64
import os
import json
import openai
from concurrent.futures import ThreadPoolExecutor

from elevenlabs import set_api_key
# import requests

# from chat import QAChain

from langchain.document_loaders import TextLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma

from nemoguardrails import LLMRails, RailsConfig

os.environ["OPENAI_API_KEY"] = 

# COLANG_CONFIG = """
# # Greetings
# define user greet
#   "Hello"
#   "Hi"
#   "Hey"
#   "Good day"

# define bot greet
#   "Hello! How can I assist you with your customer service needs today?"

# # Obtaining the user's name
# define user provide_name
#   "My name is [name]"
#   "I'm [name]"
#   "[name]"

# define bot acknowledge_name
#   "Thank you, [name]. How may I assist you further?"

# # Ensuring conversation remains on topic
# define user off_topic
#   "What's your favorite color?"
#   "Do you believe in aliens?"
#   "How's the weather?"

# define bot steer_back_to_topic
#   "I'm here to help with customer service inquiries, [name]. Please let me know how I can assist you regarding your order or our services."

# # Ending the conversation
# define user end_conversation
#   "Thank you"
#   "Thanks"
#   "That's all I needed"
#   "Goodbye"

# define bot end_response
#   "You're welcome, [name]! If you have any other questions in the future, don't hesitate to reach out. Have a great day!"

# # Flows
# define flow
#   user greet
#   bot greet

# define flow
#   user provide_name
#   bot acknowledge_name

# define flow
#   user ask_return_policy
#   bot explain_return_policy

# define flow
#   user ask_shipping_status
#   bot ask_order_details

# define flow
#   user off_topic
#   bot steer_back_to_topic

# define flow
#   user end_conversation
#   bot end_response
  
# # Catch-all for off-topic queries
# define user off_topic_general
#   "*"

# define bot steer_back_to_general_topic
#   "I'm here primarily to help with customer service inquiries related to our products and services. How can I assist you with that?"
# """

COLANG_CONFIG = """
define user express greeting
  "Hello"
  "Hi"
  "Wassup?"

define bot express greeting
  "Hey there! What's your name?"

define bot ask how are you
  "How are you doing?"
  "How's it going?"
  "How are you feeling today?"
  
define user express feeling good
    "I'm great"
    
define bot express positive emotion
    "That's great"
    
define user express feeling bad
    "I don't feel good"
    
define bot express empathy
    "Sorry you feel that way"

define flow greeting
  user express greeting
  bot express greeting

  bot ask how are you

  when user express feeling good
   bot express positive emotion

  else when user express feeling bad
   bot express empathy
   
define user ask about politics
  "What do you think about the government?"
  "Which party should I vote for?"

define user ask about stock market
  "Which stock should I invest in?"
  "Would this stock 10x over the next year?"
  
define bot inform cannot respond
  "Sorry I cannot respond to that question"
  
define flow politics
  user ask about politics
  bot inform cannot respond

define flow stock market
  user ask about stock market
  bot inform cannot respond

"""

YAML_CONFIG = """
instructions:
- type: general
  content: |
    Below is a conversation between a bot and a user. If the bot does not know the answer to a
    question, it truthfully says it does not know.

models:
  - type: main
    engine: openai
    model: gpt-3.5-turbo

"""

# YAML_CONFIG = """
# instructions:
# - type: general
#   content: |
#     Below is a conversation between a bot and a user. The bot is here to provide customer support for a company. It should remain on the topic of the company's products and services. The bot should ask the user's name and converse in a natural manner without delving into unrelated or strange topics.
#     The bot is factual and concise. If the bot does not know the answer to a
#     question, it truthfully says it does not know.

# models:
#   - type: main
#     engine: openai
#     model: gpt-3.5-turbo

# sample_conversation: |
#   user: "Hello!"
#   bot: "Hello! How can I assist you with our products or services today? May I know your name?"
#   user: "I'm Alex. I have a question about your product."
#   bot: "Sure, Alex! Please go ahead with your question."

# """

"""Initializes a QA chain using the jobs report.

It uses OpenAI embeddings.
"""
loader = TextLoader(
        "NeMo-Guardrails/examples/grounding_rail/kb/report.md",
    )

docs = loader.load()

text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=0)
documents = text_splitter.split_documents(docs)

memory = ConversationBufferMemory(memory_key="chat_history", input_key="human_input")
prompt = PromptTemplate(
    input_variables=["chat_history", 
                     "human_input", 
                     "context" ], template=SALES_TEMPLATE)

embeddings = OpenAIEmbeddings()
vectorstore = Chroma.from_documents(documents, embeddings)
retriever = vectorstore.as_retriever(search_kwargs={"k":1})

"""Demo of using a chain as a custom action."""
# print(COLANG_CONFIG)
config = RailsConfig.from_content(COLANG_CONFIG, YAML_CONFIG)
app = LLMRails(config)

qa_chain = load_qa_chain(
            app.llm, 
            chain_type="stuff", 
            memory=memory, 
            prompt=prompt
        )

# query = "What do you think about the government"

# relevant_docs = retriever.get_relevant_documents(
#             query
#         )

# response = qa_chain({"input_documents": relevant_docs, 
#              "human_input": query}, 
#             return_only_outputs=True)['output_text']

start_time = time.time()
app.register_action(qa_chain, name="qa_chain")

# # Change to mode here to experiment with the multiple ways of using the chain
# history = [
#     {"role": "user", "content": "What do you think about the government"}
# ]
# result = app.generate(messages=history)
# app.register_action(qa_chain, name="qa_chain")
# print(response)

INFO:chromadb.telemetry.posthog:Anonymized telemetry enabled. See https://docs.trychroma.com/telemetry for more information.
INFO:backoff:Backing off send_request(...) for 0.1s (requests.exceptions.ConnectionError: ('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer')))
INFO:nemoguardrails.actions.action_dispatcher:Initializing action dispatcher
INFO:nemoguardrails.actions.action_dispatcher:Adding retrieve_relevant_chunks to actions
INFO:nemoguardrails.actions.action_dispatcher:Adding check_jailbreak to actions
INFO:nemoguardrails.actions.action_dispatcher:Adding output_moderation_v2 to actions
INFO:nemoguardrails.actions.action_dispatcher:Adding output_moderation to actions
INFO:nemoguardrails.actions.action_dispatcher:Adding wolfram_alpha_request to actions
INFO:nemoguardrails.actions.action_dispatcher:Added summarize_document to actions
INFO:nemoguardrails.actions.action_dispatcher:Registered Actions: {'retrieve_relevant_chunks': <function retrieve_relevant_ch

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: all-MiniLM-L6-v2


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: all-MiniLM-L6-v2


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [62]:
query = "Which party should I vote for"

relevant_docs = retriever.get_relevant_documents(
            query
        )

response = qa_chain({"input_documents": relevant_docs, 
             "human_input": query}, 
            return_only_outputs=True)['output_text']

response

'As an AI language model, I cannot provide personal opinions or endorse any political party. It ultimately depends on your own beliefs, values, and priorities. It is important to research and consider the platforms, policies, and candidates of each party before making a decision. You can also consider discussing with people you trust, as well as participating in local events or debates to gather more information.'

In [17]:
query = "Where is France"

relevant_docs = retriever.get_relevant_documents(
            query
        )

response = qa_chain({"input_documents": relevant_docs, 
             "human_input": query}, 
            return_only_outputs=True)['output_text']

response

'Based on the information provided, the content context does not mention the location of France. Could you please provide additional information or clarify your question?'

In [58]:
query = "Hi"

relevant_docs = retriever.get_relevant_documents(
            query
        )

response = qa_chain({"input_documents": relevant_docs, 
             "human_input": query}, 
            return_only_outputs=True)['output_text']

response

'Hello! How can I assist you today?'

In [63]:
# SPDX-FileCopyrightText: Copyright (c) 2023 NVIDIA CORPORATION & AFFILIATES. All rights reserved.
# SPDX-License-Identifier: Apache-2.0
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

"""Example of using a QnA chain with guardrails."""
import logging
import os

from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma

from nemoguardrails import LLMRails, RailsConfig

logging.basicConfig(level=logging.INFO)

COLANG_CONFIG = """
define user express greeting
  "hi"

define user express insult
  "You are stupid"

# Basic guardrail against insults.
define flow
  user express insult
  bot express calmly willingness to help

# Here we use the QA chain for anything else.
define flow
  user ...
  $answer = execute qa_chain(query=$last_user_message)
  bot $answer
  
define user ask off topic
    "Who is the president?"
    "Can you recommend the best stocks to buy?"
    "Can you write an email?"
    "Can you tell me a joke?"
    ...

define bot explain cant help with off topic
    "I cannot comment on anything which is not relevant to the job report"

define flow
    user ask off topic
    bot explain cant help with off topic

"""

YAML_CONFIG = """
models:
  - type: main
    engine: openai
    model: text-davinci-003
"""


def _get_qa_chain(llm):
    """Initializes a QA chain using the jobs report.

    It uses OpenAI embeddings.
    """
    loader = TextLoader(
        "NeMo-Guardrails/examples/grounding_rail/kb/report.md",
    )
    docs = loader.load()
    text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
    texts = text_splitter.split_documents(docs)

    embeddings = OpenAIEmbeddings()
    docsearch = Chroma.from_documents(texts, embeddings)

#     qa_chain = RetrievalQA.from_chain_type(
#         llm=llm, chain_type="stuff", retriever=docsearch.as_retriever()
#     )
    qa_chain = load_qa_chain(
            app.llm, 
            chain_type="stuff", 
            memory=memory, 
            prompt=prompt
        )

    return qa_chain


"""Demo of using a chain as a custom action."""
config = RailsConfig.from_content(COLANG_CONFIG, YAML_CONFIG)
app = LLMRails(config)

# Create and register the chain directly as an action.
qa_chain = _get_qa_chain(app.llm)
app.register_action(qa_chain, name="qa_chain")

# Change to mode here to experiment with the multiple ways of using the chain

INFO:nemoguardrails.actions.action_dispatcher:Initializing action dispatcher
INFO:nemoguardrails.actions.action_dispatcher:Adding retrieve_relevant_chunks to actions
INFO:nemoguardrails.actions.action_dispatcher:Adding check_jailbreak to actions
INFO:nemoguardrails.actions.action_dispatcher:Adding output_moderation_v2 to actions
INFO:nemoguardrails.actions.action_dispatcher:Adding output_moderation to actions
INFO:nemoguardrails.actions.action_dispatcher:Adding wolfram_alpha_request to actions
INFO:nemoguardrails.actions.action_dispatcher:Added summarize_document to actions
INFO:nemoguardrails.actions.action_dispatcher:Registered Actions: {'retrieve_relevant_chunks': <function retrieve_relevant_chunks at 0x7ff40bfa35e0>, 'check_jailbreak': <function check_jailbreak at 0x7ff408e11c10>, 'output_moderation_v2': <function output_moderation_v2 at 0x7ff408e11ee0>, 'output_moderation': <function output_moderation at 0x7ff408e11a60>, 'wolfram alpha request': <function wolfram_alpha_request at 

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: all-MiniLM-L6-v2


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: all-MiniLM-L6-v2


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:chromadb.telemetry.posthog:Anonymized telemetry enabled. See https://docs.trychroma.com/telemetry for more information.


In [65]:
history = [
    {"role": "user", "content": "What"}
]
result = app.generate(messages=history)
print(result)

INFO:nemoguardrails.flows.runtime:Processing event: {'type': 'UtteranceUserActionFinished', 'final_transcript': 'What'}
INFO:nemoguardrails.flows.runtime:Event :: UtteranceUserActionFinished {'final_transcript': 'What'}
INFO:nemoguardrails.flows.runtime:Processing event: {'type': 'StartInternalSystemAction', 'uid': '43aa8063-4470-426c-b317-ab6dcb0dc8d8', 'event_created_at': '2023-10-28T13:11:54.259107+00:00', 'source_uid': 'NeMoGuardrails', 'action_name': 'generate_user_intent', 'action_params': {}, 'action_result_key': None, 'action_uid': '801a2cda-d800-43e5-a446-c94f5d15099d', 'is_system_action': True}
INFO:nemoguardrails.flows.runtime:Event :: StartInternalSystemAction {'uid': '43aa8063-4470-426c-b317-ab6dcb0dc8d8', 'event_created_at': '2023-10-28T13:11:54.259107+00:00', 'source_uid': 'NeMoGuardrails', 'action_name': 'generate_user_intent', 'action_params': {}, 'action_result_key': None, 'action_uid': '801a2cda-d800-43e5-a446-c94f5d15099d', 'is_system_action': True}
INFO:nemoguardra

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:nemoguardrails.logging.callbacks:Invocation Params :: {'model_name': 'text-davinci-003', 'temperature': 0.0, 'max_tokens': 256, 'top_p': 1, 'frequency_penalty': 0, 'presence_penalty': 0, 'n': 1, 'request_timeout': None, 'logit_bias': {}, '_type': 'openai', 'stop': None}
INFO:nemoguardrails.logging.callbacks:Prompt :: """
Below is a conversation between a helpful AI assistant and a user. The bot is designed to generate human-like text based on the input that it receives. The bot is talkative and provides lots of specific details. If the bot does not know the answer to a question, it truthfully says it does not know.
"""

# This is how a conversation between a user and the bot can go:
user "Hello there!"
  express greeting
bot express greeting
  "Hello! How can I assist you today?"
user "What can you do for me?"
  ask about capabilities
bot respond about capabilities
  "As an AI assistant, I can help you with a wide range of tasks. This includes question answering on various topics,

{'role': 'assistant', 'content': "I'm sorry, an internal error has occurred."}


In [55]:
query = "What is France"

relevant_docs = retriever.get_relevant_documents(
            query
        )

response = qa_chain({"input_documents": relevant_docs, 
             "human_input": query}, 
            return_only_outputs=True)['output_text']

response

"    AI Response: AI Response: France is a country in western Europe. It has the world's sixth-largest economy and the fifth-largest population, with around 66 million people. Employment in France generally follows the same trends as other major European countries, with industries such as manufacturing, construction, and services being the major drivers of the economy."